# Water Mass Analysis

#### This notebook performs the following:

1) Loads transect glider dataframes from 01 notebook <br>
2) <br>

## Imports

In [1]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import math
import xarray as xr
import gsw
import sys
import pyompa
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import scipy.interpolate as interp
from contextlib import contextmanager,redirect_stderr,redirect_stdout
from os import devnull
import time
import datetime

## For supressing the OMP output
@contextmanager
def suppress_stdout_stderr():
    """A context manager that redirects stdout and stderr to devnull"""
    with open(devnull, 'w') as fnull:
        with redirect_stderr(fnull) as err, redirect_stdout(fnull) as out:
            yield (err, out)


## To import functions from Slocum-AD2CP GitHub repository, make this path the path to where the repo exists locally
sys.path.insert(0,'/Users/joegradone/SynologyDrive/Drive/Rutgers/Research/code/GitHub/Slocum-AD2CP/src/analysis/')
from analysis import gsw_rho, profile_mld, get_erddap_dataset, grid_glider_data, dist_from_lat_lon

## Set some plotting formats
plt.style.use('seaborn-poster')
myFmtshort = mdates.DateFormatter('%m/%d\n%H:%M')
myFmtlong = mdates.DateFormatter('%m/%d/%y \n%H:%M')
myFmt = mdates.DateFormatter('%m/%d/%y')


###################################################################################################################
######################################### Setting for water mass analysis #########################################
###################################################################################################################

#Define a parameter to represent remineralization in terms of phosphate
convertedparamgroups = [
    pyompa.ConvertedParamGroup(
        groupname="phosphate_remin",
        conversion_ratios=[{"oxygen_concentration": -170, "phosphate": 1.0}],
        always_positive=True)
]

paramweightings = {
    "conservative_temperature": 1.0,
    "absolute_salinity": 1.0,
    "mass": 1.0,
}

settings = {
    "param_names": ["conservative_temperature", "absolute_salinity", "mass"],
    "convertedparam_groups": convertedparamgroups,
    "param_weightings": paramweightings,
}



ModuleNotFoundError: No module named 'pyompa'

## Load transect data

In [ ]:
dom1_22 = xr.open_dataset('./data/Dominica_Transect_1_2022.nc')
dom2_22 = xr.open_dataset('./data/Dominica_Transect_2_2022.nc')
dom1_23 = xr.open_dataset('./data/Dominica_Transect_1_2023.nc')
dom2_23 = xr.open_dataset('./data/Dominica_Transect_2_2023.nc')

stluc1_22 = xr.open_dataset('./data/St_Lucia_Transect_1_2022.nc')
stluc2_22 = xr.open_dataset('./data/St_Lucia_Transect_2_2022.nc')
stluc1_23 = xr.open_dataset('./data/St_Lucia_Transect_1_2023.nc')
stluc2_23 = xr.open_dataset('./data/St_Lucia_Transect_2_2023.nc')


stvin1_22 = xr.open_dataset('./data/St_Vincent_Transect_1_2022.nc')
stvin2_22 = xr.open_dataset('./data/St_Vincent_Transect_2_2022.nc')
stvin1_23 = xr.open_dataset('./data/St_Vincent_Transect_1_2023.nc')
stvin2_23 = xr.open_dataset('./data/St_Vincent_Transect_2_2023.nc')
stvin2_23

## Load WOA data

In [ ]:
temp_ds = xr.open_dataset('https://www.ncei.noaa.gov/thredds-ocean/dodsC/ncei/woa/temperature/decav/1.00/woa18_decav_t00_01.nc', decode_times=False)
sal_ds  = xr.open_dataset('https://www.ncei.noaa.gov/thredds-ocean/dodsC/ncei/woa/salinity/decav/1.00/woa18_decav_s00_01.nc', decode_times=False)
temp_ds

## Initialize endmemberdf

In [ ]:
## initialize
NAW =       ["NAW",   np.nan,   np.nan]
SAW =       ["SAW",   np.nan,  np.nan]


def prepare_endmember_df(endmembers_arr):
    df = pd.DataFrame(data=endmembers_arr,
                      columns=["endmember_name", "conservative_temperature", "absolute_salinity"])
    df["mass"] = 1
    return df

endmemberdf = prepare_endmember_df([NAW, SAW])
endmemberdf

## Subset for SAW and NAW

In [ ]:
###################################################################################################################
### Upper NAW ###
min_lon = -60
max_lon = -40
min_lat = 13
max_lat = 17

## subset
naw_temp_ds1 = temp_ds.sel(lat=slice(min_lat,max_lat),lon=slice(min_lon,max_lon),time=temp_ds.time[0])
naw_sal_ds1  = sal_ds.sel(lat=slice(min_lat,max_lat),lon=slice(min_lon,max_lon),time=sal_ds.time[0])

## average profile
naw_mean_temp1 = np.nanmax(naw_temp_ds1.t_mn, axis=(1,2))
naw_mean_sal1  = np.nanmax(naw_sal_ds1.s_mn, axis=(1,2))

## convert to SA and PT and calculate sigma0
naw_mean_sal1 = gsw.SA_from_SP(naw_mean_sal1, naw_sal_ds1.depth.values, -40, -30)
naw_mean_temp1 = gsw.pt0_from_t(naw_mean_sal1, naw_mean_temp1, naw_sal_ds1.depth.values)
naw_mean_den1 = gsw.sigma0(naw_mean_sal1,naw_mean_temp1)+1000

## regular depth grid for all so just use this, do this before subsetting naw_mean_den1
mean_depth = naw_sal_ds1.depth.values[np.where((naw_mean_den1>=1024.5) & (naw_mean_den1 <=1027.6))]

## subset by density
naw_den1_ind = np.where((naw_mean_den1>=1024.5) & (naw_mean_den1 <=1026.3))
naw_mean_sal1 = naw_mean_sal1[naw_den1_ind]
naw_mean_temp1 = naw_mean_temp1[naw_den1_ind]
naw_mean_den1 = naw_mean_den1[naw_den1_ind]

###################################################################################################################
### Lower NAW ###
min_lon = -60
max_lon = -40
min_lat = 23
max_lat = 26

## subset
naw_temp_ds2 = temp_ds.sel(lat=slice(min_lat,max_lat),lon=slice(min_lon,max_lon),time=temp_ds.time[0])
naw_sal_ds2  = sal_ds.sel(lat=slice(min_lat,max_lat),lon=slice(min_lon,max_lon),time=sal_ds.time[0])

## average profile
naw_mean_temp2 = np.nanmean(naw_temp_ds2.t_mn, axis=(1,2))
naw_mean_sal2  = np.nanmean(naw_sal_ds2.s_mn, axis=(1,2))

## convert to SA and PT and calculate sigma0
naw_mean_sal2 = gsw.SA_from_SP(naw_mean_sal2, naw_sal_ds2.depth.values, -40, -30)
naw_mean_temp2 = gsw.pt0_from_t(naw_mean_sal2, naw_mean_temp2, naw_sal_ds2.depth.values)
naw_mean_den2 = gsw.sigma0(naw_mean_sal2,naw_mean_temp2)+1000

## subset by density
naw_den2_ind = np.where((naw_mean_den2>=1026.3) & (naw_mean_den2 <=1027.6))
naw_mean_sal2 = naw_mean_sal2[naw_den2_ind]
naw_mean_temp2 = naw_mean_temp2[naw_den2_ind]
naw_mean_den2 = naw_mean_den2[naw_den2_ind]

naw_mean_temp = np.concatenate((naw_mean_temp1,naw_mean_temp2))
naw_mean_sal = np.concatenate((naw_mean_sal1,naw_mean_sal2))
naw_mean_den = gsw.sigma0(naw_mean_sal,naw_mean_temp)+1000

###################################################################################################################
###################################################################################################################
### upper SAW ###
min_lon = 0
max_lon = 10
min_lat = -12
max_lat = -8

## subset
saw_temp_ds1 = temp_ds.sel(lat=slice(min_lat,max_lat),lon=slice(min_lon,max_lon),time=temp_ds.time[0])
saw_sal_ds1  = sal_ds.sel(lat=slice(min_lat,max_lat),lon=slice(min_lon,max_lon),time=sal_ds.time[0])

## Make average profiles
saw_mean_temp1 = np.nanmean(saw_temp_ds1.t_mn, axis=(1,2))
saw_mean_sal1  = np.nanmean(saw_sal_ds1.s_mn, axis=(1,2))

## convert to SA and PT and calculate sigma0
saw_mean_sal1 = gsw.SA_from_SP(saw_mean_sal1, saw_sal_ds1.depth.values, -40, -30)
saw_mean_temp1 = gsw.pt0_from_t(saw_mean_sal1, saw_mean_temp1, saw_sal_ds1.depth.values)
saw_mean_den1 = gsw.sigma0(saw_mean_sal1,saw_mean_temp1)+1000

## subset by density
saw_den1_ind = np.where((saw_mean_den1>=1024.5) & (saw_mean_den1 <=1026.3))
saw_mean_sal1 = saw_mean_sal1[saw_den1_ind]
saw_mean_temp1 = saw_mean_temp1[saw_den1_ind]
saw_mean_den1 = saw_mean_den1[saw_den1_ind]

###################################################################################################################
### lower SAW ###
min_lon = -32
max_lon = -28
min_lat = -12
max_lat = -8

## subset
saw_temp_ds2 = temp_ds.sel(lat=slice(min_lat,max_lat),lon=slice(min_lon,max_lon),time=temp_ds.time[0])
saw_sal_ds2  = sal_ds.sel(lat=slice(min_lat,max_lat),lon=slice(min_lon,max_lon),time=sal_ds.time[0])

## Make average profiles
saw_mean_temp2 = np.nanmean(saw_temp_ds2.t_mn, axis=(1,2))
saw_mean_sal2  = np.nanmean(saw_sal_ds2.s_mn, axis=(1,2))

## convert to SA and PT and calculate sigma0
saw_mean_sal2 = gsw.SA_from_SP(saw_mean_sal2, saw_sal_ds2.depth.values, -40, -30)
saw_mean_temp2 = gsw.pt0_from_t(saw_mean_sal2, saw_mean_temp2, saw_sal_ds2.depth.values)
saw_mean_den2 = gsw.sigma0(saw_mean_sal2,saw_mean_temp2)+1000

## subset by density
saw_den2_ind = np.where((saw_mean_den2>=1026.3) & (saw_mean_den2 <=1027.6))
saw_mean_sal2 = saw_mean_sal2[saw_den2_ind]
saw_mean_temp2 = saw_mean_temp2[saw_den2_ind]
saw_mean_den2 = saw_mean_den2[saw_den2_ind]

saw_mean_temp = np.concatenate((saw_mean_temp1,saw_mean_temp2))
saw_mean_sal = np.concatenate((saw_mean_sal1,saw_mean_sal2))
saw_mean_den = gsw.sigma0(saw_mean_sal,saw_mean_temp)+1000



## Smooth
griddepth = np.arange(0,1001,2)

naw_interp_mean_temp =  interp.griddata(mean_depth,naw_mean_temp,griddepth)
naw_interp_mean_sal =  interp.griddata(mean_depth,naw_mean_sal,griddepth)
naw_interp_mean_den =  interp.griddata(mean_depth,naw_mean_den,griddepth)

saw_interp_mean_temp =  interp.griddata(mean_depth,saw_mean_temp,griddepth)
saw_interp_mean_sal =  interp.griddata(mean_depth,saw_mean_sal,griddepth)
saw_interp_mean_den =  interp.griddata(mean_depth,saw_mean_den,griddepth)

## Save
d = {'depth': griddepth, 'naw_temp':naw_interp_mean_temp, 'naw_sal': naw_interp_mean_sal,
     'naw_den': naw_interp_mean_den, 'saw_temp': saw_interp_mean_temp, 'saw_sal': saw_interp_mean_sal, 'saw_den':saw_interp_mean_den}

df = pd.DataFrame(data=d)

df.to_csv('./data/naw_saw_endmember_t_s_d_profile.csv')




## Water Mass Analysis

In [ ]:

def solve_endmember_fractions_single(df,naw_interp_mean_temp,naw_interp_mean_sal,naw_interp_mean_den, saw_interp_mean_temp,saw_interp_mean_sal,saw_interp_mean_den):
    ## preallocation soln matrix
    naw_frac_df = np.empty(df.temperature.shape)
    naw_frac_df[:] = np.nan
    saw_frac_df = np.empty(df.temperature.shape)
    saw_frac_df[:] = np.nan
    temp_resid_df = np.empty(df.temperature.shape)
    temp_resid_df[:] = np.nan
    sal_resid_df = np.empty(df.temperature.shape)
    sal_resid_df[:] = np.nan
    mass_resid_df = np.empty(df.temperature.shape)
    mass_resid_df[:] = np.nan

    ## Loop through each row (depth) and perform water mass analysis
    for x in np.arange(0,len(df.depth)):
        for y in np.arange(0,len(df.latitude)):
            ## if there is a nan value, skip
            if ~np.isnan(df.temperature.values[x,y]):
                ## if density is less than 1024.5 set SAW = 1
                if df.density.values[x,y] <= 1024.5:
                    ## save soln'n
                    naw_frac_df[x,y]   = 0
                    saw_frac_df[x,y]   = 1
                    temp_resid_df[x,y] = 0
                    sal_resid_df[x,y]  = 0
                    mass_resid_df[x,y] = 0
                else:
                    ## subset to just the depth row of interest and make pandas dataframe
                    d = {'conservative_temperature': df.temperature.values[x,y], 'absolute_salinity': df.salinity.values[x,y]}
                    obs_df = pd.DataFrame(data=d, index=[0])
                    obs_df["mass"] = 1.0

                    ## find matching closest isopycnal index in source water data
                    ## theoretically a different index for NAW vs SAW
                    diffs = np.abs(df.density.values[x,y]-naw_interp_mean_den)
                    naw_iso_ind = np.nanargmin(diffs)

                    diffs = np.abs(df.density.values[x,y]-saw_interp_mean_den)
                    saw_iso_ind = np.nanargmin(diffs)

                    ## oull out NAW and SAW temp and sal
                    endmemberdf['conservative_temperature'][0] = naw_interp_mean_temp[naw_iso_ind]
                    endmemberdf['conservative_temperature'][1] = saw_interp_mean_temp[saw_iso_ind]
                    endmemberdf['absolute_salinity'][0] = naw_interp_mean_sal[naw_iso_ind]
                    endmemberdf['absolute_salinity'][1] = saw_interp_mean_sal[saw_iso_ind]
                    ## run analysis
                    ompa_soln = pyompa.OMPAProblem(obs_df = obs_df, **settings).solve(endmemberdf,endmember_name_column = "endmember_name")
                    ## save soln'n
                    naw_frac_df[x,y]   = ompa_soln.endmember_fractions[0,0]
                    saw_frac_df[x,y]   = ompa_soln.endmember_fractions[0,1]
                    temp_resid_df[x,y] = ompa_soln.param_residuals[0,0]
                    sal_resid_df[x,y]  = ompa_soln.param_residuals[0,1]
                    mass_resid_df[x,y] = ompa_soln.param_residuals[0,2]

    df["naw_frac"]=(['depth', 'latitude'],  naw_frac_df)
    df["saw_frac"]=(['depth', 'latitude'],  saw_frac_df)
    df["temp_resid"]=(['depth', 'latitude'],  temp_resid_df)
    df["sal_resid"]=(['depth', 'latitude'],  sal_resid_df)
    df["mass_resid"]=(['depth', 'latitude'],  mass_resid_df)

    ############ Now calculate transport!!!!!
    ## preallocate for loop
    transport = np.empty(df.VgeoEW.shape)
    transport[:] = np.NaN
    ## calc dz based on depth data
    dz = df.depth.values[1]-df.depth.values[0]

    ## calculate transport in Sv based on dz and dy
    transport = (df.VgeoEW.values*df.attrs['geo_dz']*df.attrs['dy'])/(10**6)

    ## now multiply by %naw oand %saw to calculate transport of naw and saw
    naw_transport = transport*naw_frac_df
    saw_transport = transport*saw_frac_df

    df["transport"]=(['depth', 'latitude'],  transport)
    df["naw_transport"]=(['depth', 'latitude'],  naw_transport)
    df["saw_transport"]=(['depth', 'latitude'],  saw_transport)
    return df


In [ ]:
with suppress_stdout_stderr():
    dom1_22_final = solve_endmember_fractions_single(dom1_22,naw_interp_mean_temp,naw_interp_mean_sal,naw_interp_mean_den,saw_interp_mean_temp,saw_interp_mean_sal,saw_interp_mean_den)
    dom2_22_final = solve_endmember_fractions_single(dom2_22,naw_interp_mean_temp,naw_interp_mean_sal,naw_interp_mean_den,saw_interp_mean_temp,saw_interp_mean_sal,saw_interp_mean_den)
    dom1_23_final = solve_endmember_fractions_single(dom1_23,naw_interp_mean_temp,naw_interp_mean_sal,naw_interp_mean_den,saw_interp_mean_temp,saw_interp_mean_sal,saw_interp_mean_den)
    dom2_23_final = solve_endmember_fractions_single(dom2_23,naw_interp_mean_temp,naw_interp_mean_sal,naw_interp_mean_den,saw_interp_mean_temp,saw_interp_mean_sal,saw_interp_mean_den)
    
    stluc1_22_final = solve_endmember_fractions_single(stluc1_22,naw_interp_mean_temp,naw_interp_mean_sal,naw_interp_mean_den,saw_interp_mean_temp,saw_interp_mean_sal,saw_interp_mean_den)
    stluc2_22_final = solve_endmember_fractions_single(stluc2_22,naw_interp_mean_temp,naw_interp_mean_sal,naw_interp_mean_den,saw_interp_mean_temp,saw_interp_mean_sal,saw_interp_mean_den)
    stluc1_23_final = solve_endmember_fractions_single(stluc1_23,naw_interp_mean_temp,naw_interp_mean_sal,naw_interp_mean_den,saw_interp_mean_temp,saw_interp_mean_sal,saw_interp_mean_den)
    stluc2_23_final = solve_endmember_fractions_single(stluc2_23,naw_interp_mean_temp,naw_interp_mean_sal,naw_interp_mean_den,saw_interp_mean_temp,saw_interp_mean_sal,saw_interp_mean_den)
    
    stvin1_22_final = solve_endmember_fractions_single(stvin1_22,naw_interp_mean_temp,naw_interp_mean_sal,naw_interp_mean_den,saw_interp_mean_temp,saw_interp_mean_sal,saw_interp_mean_den)
    stvin2_22_final = solve_endmember_fractions_single(stvin2_22,naw_interp_mean_temp,naw_interp_mean_sal,naw_interp_mean_den,saw_interp_mean_temp,saw_interp_mean_sal,saw_interp_mean_den)
    stvin1_23_final = solve_endmember_fractions_single(stvin1_23,naw_interp_mean_temp,naw_interp_mean_sal,naw_interp_mean_den,saw_interp_mean_temp,saw_interp_mean_sal,saw_interp_mean_den)
    stvin2_23_final = solve_endmember_fractions_single(stvin2_23,naw_interp_mean_temp,naw_interp_mean_sal,naw_interp_mean_den,saw_interp_mean_temp,saw_interp_mean_sal,saw_interp_mean_den)

dom1_22_final

In [ ]:
plt.figure(figsize=(15,10))
plt.contourf(dom1_22_final.latitude,dom1_22_final.depth,dom1_22_final.saw_frac,levels=np.arange(0,1.01,0.025))
plt.ylim(1000,0)
cbar = plt.colorbar()
plt.ylabel('Depth [m]',size=18)
plt.xlabel('Latitude',size=18)
plt.title('Dominica Transect #1 Nov-22',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=18)
cbar.set_label('% SAW',size=18)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1])
cbar.set_ticklabels([0,20,40,60,80,100])
cbar.ax.tick_params(labelsize=18) 


In [ ]:
plt.figure(figsize=(15,10))
plt.contourf(dom2_22_final.latitude,dom2_22_final.depth,dom2_22_final.saw_frac,levels=np.arange(0,1.01,0.025))
plt.ylim(1000,0)
cbar = plt.colorbar()
plt.ylabel('Depth [m]',size=18)
plt.xlabel('Latitude',size=18)
plt.title('Dominica Transect #2 Nov-22',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=18)
cbar.set_label('% SAW',size=18)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1])
cbar.set_ticklabels([0,20,40,60,80,100])
cbar.ax.tick_params(labelsize=18) 


In [ ]:
plt.figure(figsize=(15,10))
plt.contourf(dom1_23_final.latitude,dom1_23_final.depth,dom1_23_final.saw_frac,levels=np.arange(0,1.01,0.025))
plt.ylim(1000,0)
cbar = plt.colorbar()
plt.ylabel('Depth [m]',size=18)
plt.xlabel('Latitude',size=18)
plt.title('Dominica Transect #1 Jun-23',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=18)
cbar.set_label('% SAW',size=18)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1])
cbar.set_ticklabels([0,20,40,60,80,100])
cbar.ax.tick_params(labelsize=18) 


In [ ]:
plt.figure(figsize=(15,10))
plt.contourf(dom2_23_final.latitude,dom2_23_final.depth,dom2_23_final.saw_frac,levels=np.arange(0,1.01,0.025))
plt.ylim(1000,0)
cbar = plt.colorbar()
plt.ylabel('Depth [m]',size=18)
plt.xlabel('Latitude',size=18)
plt.title('Dominica Transect #2 Jun-23',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=18)
cbar.set_label('% SAW',size=18)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1])
cbar.set_ticklabels([0,20,40,60,80,100])
cbar.ax.tick_params(labelsize=18) 


In [ ]:
plt.figure(figsize=(15,10))
plt.contourf(stluc1_22_final.latitude,stluc1_22_final.depth,stluc1_22_final.saw_frac,levels=np.arange(0,1.01,0.025))
plt.ylim(1000,0)
cbar = plt.colorbar()
plt.ylabel('Depth [m]',size=18)
plt.xlabel('Latitude',size=18)
plt.title('St. Lucia Transect #1 Nov-22',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=18)
cbar.set_label('% SAW',size=18)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1])
cbar.set_ticklabels([0,20,40,60,80,100])
cbar.ax.tick_params(labelsize=18) 


In [ ]:
plt.figure(figsize=(15,10))
plt.contourf(stluc2_22_final.latitude,stluc2_22_final.depth,stluc2_22_final.saw_frac,levels=np.arange(0,1.01,0.025))
plt.ylim(1000,0)
cbar = plt.colorbar()
plt.ylabel('Depth [m]',size=18)
plt.xlabel('Latitude',size=18)
plt.title('St. Lucia Transect #2 Nov-22',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=18)
cbar.set_label('% SAW',size=18)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1])
cbar.set_ticklabels([0,20,40,60,80,100])
cbar.ax.tick_params(labelsize=18) 


In [ ]:
plt.figure(figsize=(15,10))
plt.contourf(stluc1_23_final.latitude,stluc1_23_final.depth,stluc1_23_final.saw_frac,levels=np.arange(0,1.01,0.025))
plt.ylim(1000,0)
cbar = plt.colorbar()
plt.ylabel('Depth [m]',size=18)
plt.xlabel('Latitude',size=18)
plt.title('St. Lucia Transect #1 Jun-23',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=18)
cbar.set_label('% SAW',size=18)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1])
cbar.set_ticklabels([0,20,40,60,80,100])
cbar.ax.tick_params(labelsize=18) 


In [ ]:
plt.figure(figsize=(15,10))
plt.contourf(stluc2_23_final.latitude,stluc2_23_final.depth,stluc2_23_final.saw_frac,levels=np.arange(0,1.01,0.025))
plt.ylim(1000,0)
cbar = plt.colorbar()
plt.ylabel('Depth [m]',size=18)
plt.xlabel('Latitude',size=18)
plt.title('St. Lucia Transect #2 Jun-23',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=18)
cbar.set_label('% SAW',size=18)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1])
cbar.set_ticklabels([0,20,40,60,80,100])
cbar.ax.tick_params(labelsize=18) 


In [ ]:
plt.figure(figsize=(15,10))
plt.contourf(stvin1_22_final.latitude,stvin1_22_final.depth,stvin1_22_final.saw_frac,levels=np.arange(0,1.01,0.025))
plt.ylim(1000,0)
cbar = plt.colorbar()
plt.ylabel('Depth [m]',size=18)
plt.xlabel('Latitude',size=18)
plt.title('St. Vincent Transect #1 Nov-22',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=18)
cbar.set_label('% SAW',size=18)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1])
cbar.set_ticklabels([0,20,40,60,80,100])
cbar.ax.tick_params(labelsize=18) 


In [ ]:
plt.figure(figsize=(15,10))
plt.contourf(stvin2_22_final.latitude,stvin2_22_final.depth,stvin2_22_final.saw_frac,levels=np.arange(0,1.01,0.025))
plt.ylim(1000,0)
cbar = plt.colorbar()
plt.ylabel('Depth [m]',size=18)
plt.xlabel('Latitude',size=18)
plt.title('St. Vincent Transect #2 Nov-22',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=18)
cbar.set_label('% SAW',size=18)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1])
cbar.set_ticklabels([0,20,40,60,80,100])
cbar.ax.tick_params(labelsize=18) 


In [ ]:
plt.figure(figsize=(15,10))
plt.contourf(stvin1_23_final.latitude,stvin1_23_final.depth,stvin1_23_final.saw_frac,levels=np.arange(0,1.01,0.025))
plt.ylim(1000,0)
cbar = plt.colorbar()
plt.ylabel('Depth [m]',size=18)
plt.xlabel('Latitude',size=18)
plt.title('St. Vincent Transect #1 Jun-23',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=18)
cbar.set_label('% SAW',size=18)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1])
cbar.set_ticklabels([0,20,40,60,80,100])
cbar.ax.tick_params(labelsize=18) 


In [ ]:
plt.figure(figsize=(15,10))
plt.contourf(stvin2_23_final.latitude,stvin2_23_final.depth,stvin2_23_final.saw_frac,levels=np.arange(0,1.01,0.025))
plt.ylim(1000,0)
cbar = plt.colorbar()
plt.ylabel('Depth [m]',size=18)
plt.xlabel('Latitude',size=18)
plt.title('St. Vincent Transect #2 Jun-23',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=18)
cbar.set_label('% SAW',size=18)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1])
cbar.set_ticklabels([0,20,40,60,80,100])
cbar.ax.tick_params(labelsize=18) 


In [ ]:
plt.figure(figsize=(20,20))

fig, axs = plt.subplots(1, 2, figsize=(30, 15))


axs[0].contourf(dom1_22_final.latitude,dom1_22_final.depth,dom1_22_final.saw_frac,levels=np.arange(0,1.01,0.025))
axs[0].set_ylim(1000,0)
axs[0].set_ylabel('Depth [m]',size=18)
axs[0].set_xlabel('Latitude',size=18)
axs[0].set_title('Dominica 2022')

im = axs[1].contourf(dom1_23_final.latitude,dom1_23_final.depth,dom1_23_final.saw_frac,levels=np.arange(0,1.01,0.025))
axs[1].set_ylim(1000,0)
axs[1].set_xlabel('Latitude',size=18)
axs[1].set_title('Dominica 2023')
plt.gca().tick_params(axis='both', which='major', labelsize=18)


cbar = plt.colorbar(im)
cbar.set_label('% SAW',size=18)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1])
cbar.set_ticklabels([0,20,40,60,80,100])
cbar.ax.tick_params(labelsize=18) 


In [ ]:
plt.figure(figsize=(20,20))

fig, axs = plt.subplots(1, 2, figsize=(30, 15))


axs[0].contourf(stluc1_22_final.latitude,stluc1_22_final.depth,stluc1_22_final.saw_frac,levels=np.arange(0,1.01,0.025))
axs[0].set_ylim(1000,0)
axs[0].set_ylabel('Depth [m]',size=18)
axs[0].set_xlabel('Latitude',size=18)
axs[0].set_title('St. Lucia 2022')

im = axs[1].contourf(stluc1_23_final.latitude,stluc1_23_final.depth,stluc1_23_final.saw_frac,levels=np.arange(0,1.01,0.025))
axs[1].set_ylim(1000,0)
axs[1].set_xlabel('Latitude',size=18)
axs[1].set_title('St. Lucia 2023')
plt.gca().tick_params(axis='both', which='major', labelsize=18)


cbar = plt.colorbar(im)
cbar.set_label('% SAW',size=18)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1])
cbar.set_ticklabels([0,20,40,60,80,100])
cbar.ax.tick_params(labelsize=18) 


In [ ]:
plt.figure(figsize=(20,20))

fig, axs = plt.subplots(1, 2, figsize=(30, 15))


axs[0].contourf(stvin1_22_final.latitude,stvin1_22_final.depth,stvin1_22_final.saw_frac,levels=np.arange(0,1.01,0.025))
axs[0].set_ylim(1000,0)
axs[0].set_ylabel('Depth [m]',size=18)
axs[0].set_xlabel('Latitude',size=18)
axs[0].set_title('St. Vincent 2022')

im = axs[1].contourf(stvin1_23_final.latitude,stvin1_23_final.depth,stvin1_23_final.saw_frac,levels=np.arange(0,1.01,0.025))
axs[1].set_ylim(1000,0)
axs[1].set_xlabel('Latitude',size=18)
axs[1].set_title('St. Vincent 2023')
plt.gca().tick_params(axis='both', which='major', labelsize=18)


cbar = plt.colorbar(im)
cbar.set_label('% SAW',size=18)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1])
cbar.set_ticklabels([0,20,40,60,80,100])
cbar.ax.tick_params(labelsize=18) 


In [ ]:
 plt.figure(figsize=(12, 10))

font_size = 26

s = 100

## SAW
plt.scatter(df.saw_sal,df.saw_temp,s=s,label='South Atlantic')
## NAW
plt.scatter(df.naw_sal,df.naw_temp,s=s,label='North Atlantic')

## markersize
plt.scatter(dom1_22_final.salinity.values,dom1_22_final.temperature.values,s=10,c='black',label='Glider Data')
plt.scatter(dom2_22_final.salinity.values,dom2_22_final.temperature.values,s=10,c='black')
plt.scatter(dom1_23_final.salinity.values,dom1_23_final.temperature.values,s=10,c='grey')
plt.scatter(dom2_23_final.salinity.values,dom2_23_final.temperature.values,s=10,c='grey')
plt.scatter(stluc1_22_final.salinity.values,stluc1_22_final.temperature.values,s=10,c='black')
plt.scatter(stluc2_22_final.salinity.values,stluc2_22_final.temperature.values,s=10,c='black')
plt.scatter(stluc1_23_final.salinity.values,stluc1_23_final.temperature.values,s=10,c='grey')
plt.scatter(stluc2_23_final.salinity.values,stluc2_23_final.temperature.values,s=10,c='grey')
plt.scatter(stvin1_22_final.salinity.values,stvin1_22_final.temperature.values,s=10,c='black')
plt.scatter(stvin2_22_final.salinity.values,stvin2_22_final.temperature.values,s=10,c='black')
plt.scatter(stvin1_23_final.salinity.values,stvin1_23_final.temperature.values,s=10,c='grey')
plt.scatter(stvin2_23_final.salinity.values,stvin2_23_final.temperature.values,s=10,c='grey')


plt.gca().tick_params(axis='both', which='major', labelsize=font_size)
plt.ylabel('Potential Temperature [°C]',fontsize=font_size)
plt.xlabel('Salinity [PSU]',fontsize=font_size)
plt.ylim(3,31)
plt.xlim(34,37.75)

# lgnd = axs[1,1].legend(fontsize=font_size,frameon=False,loc='lower right',markerfirst=False,handletextpad=-0.25, labelspacing=0.075,borderaxespad=-0.3)
# for handle in lgnd.legendHandles:
#     handle.set_sizes([500])

######## This plots the contours
mint=1
maxt=30
mins=34
maxs=37.5
tempL=np.linspace(mint-1,maxt+1,399)
salL=np.linspace(mins-1,maxs+1,399)
Tg, Sg = np.meshgrid(tempL,salL)

z=np.linspace(-1000,0,399)
p = gsw.p_from_z(z,lat=0)

sigma_theta = gsw.sigma0(Sg, Tg)+1000

#sigma_theta = gsw.sigma0(Sg, Tg)+1000 # ignore effects of pressure on density
cnt = np.linspace(sigma_theta.min(), sigma_theta.max(),399)
ind = np.argwhere(sigma_theta > 1015.7)
st_short = sigma_theta[ind]
cs = plt.contour(Sg, Tg, sigma_theta, colors='grey', zorder=1 ,levels=[1024.5,1026.3,1026.8,1027.1,1027.6])
levels = cs.levels
manual_locations = [(34.9, 20), (34.5, 12), (37.2,18), (36.9,15), (36.65,12)]
plt.clabel(cs,levels, fontsize=16,inline=True,fmt='%.1f', manual=manual_locations)

lgnd = plt.legend(frameon=False,loc='upper left')
for handle in lgnd.legendHandles:
    handle.set_sizes([150])


plt.title('Windward Passages',fontsize=24)

#plt.savefig('/home/jg1200/Figures/Anegada_SAW_NAW_End_Members.png', bbox_inches='tight',dpi=300,facecolor='white')



In [ ]:

def calc_transport_by_water_mass(dft1,dft2):
    sw_saw1 = np.nansum(dft1.saw_transport.values[np.where(dft1.density<=1024.5)])
    sw_naw1 = np.nansum(dft1.naw_transport.values[np.where(dft1.density<=1024.5)])
    sw_saw2 = np.nansum(dft2.saw_transport.values[np.where(dft2.density<=1024.5)])
    sw_naw2 = np.nansum(dft2.naw_transport.values[np.where(dft2.density<=1024.5)])
    sw_saw = np.nanmean([sw_saw1,sw_saw2])
    sw_saw_std = np.nanstd([sw_saw1,sw_saw2])
    sw_naw = np.nanmean([sw_naw1,sw_naw2])
    sw_naw_std = np.nanstd([sw_naw1,sw_naw2])

    smw_naw1 = np.nansum(dft1.naw_transport.values[np.where((dft1.density>=1024.5) & (dft1.density<1026.3))])
    smw_saw1 = np.nansum(dft1.saw_transport.values[np.where((dft1.density>=1024.5) & (dft1.density<1026.3))])
    smw_naw2 = np.nansum(dft2.naw_transport.values[np.where((dft2.density>=1024.5) & (dft2.density<1026.3))])
    smw_saw2 = np.nansum(dft2.saw_transport.values[np.where((dft2.density>=1024.5) & (dft2.density<1026.3))])
    smw_saw = np.nanmean([smw_saw1,smw_saw2])
    smw_saw_std = np.nanstd([smw_saw1,smw_saw2])
    smw_naw = np.nanmean([smw_naw1,smw_naw2])
    smw_naw_std = np.nanstd([smw_naw1,smw_naw2])
    
    ucw_naw1 = np.nansum(dft1.naw_transport.values[np.where((dft1.density>=1026.3) & (dft1.density<1026.8))])
    ucw_saw1 = np.nansum(dft1.saw_transport.values[np.where((dft1.density>=1026.3) & (dft1.density<1026.8))])
    ucw_naw2 = np.nansum(dft2.naw_transport.values[np.where((dft2.density>=1026.3) & (dft2.density<1026.8))])
    ucw_saw2 = np.nansum(dft2.saw_transport.values[np.where((dft2.density>=1026.3) & (dft2.density<1026.8))])
    ucw_saw = np.nanmean([ucw_saw1,ucw_saw2])
    ucw_saw_std = np.nanstd([ucw_saw1,ucw_saw2])
    ucw_naw = np.nanmean([ucw_naw1,ucw_naw2])
    ucw_naw_std = np.nanstd([ucw_naw1,ucw_naw2])
        
    lcw_naw1 = np.nansum(dft1.naw_transport.values[np.where((dft1.density>=1026.8) & (dft1.density<1027.1))])
    lcw_saw1 = np.nansum(dft1.saw_transport.values[np.where((dft1.density>=1026.8) & (dft1.density<1027.1))])
    lcw_naw2 = np.nansum(dft2.naw_transport.values[np.where((dft2.density>=1026.8) & (dft2.density<1027.1))])
    lcw_saw2 = np.nansum(dft2.saw_transport.values[np.where((dft2.density>=1026.8) & (dft2.density<1027.1))])
    lcw_saw = np.nanmean([lcw_saw1,lcw_saw2])
    lcw_saw_std = np.nanstd([lcw_saw1,lcw_saw2])
    lcw_naw = np.nanmean([lcw_naw1,lcw_naw2])
    lcw_naw_std = np.nanstd([lcw_naw1,lcw_naw2])
        
    iw_naw1 = np.nansum(dft1.naw_transport.values[np.where(dft1.density>=1027.1)])
    iw_saw1 = np.nansum(dft1.saw_transport.values[np.where(dft1.density>=1027.1)])
    iw_naw2 = np.nansum(dft2.naw_transport.values[np.where(dft2.density>=1027.1)])
    iw_saw2 = np.nansum(dft2.saw_transport.values[np.where(dft2.density>=1027.1)])
    iw_saw = np.nanmean([iw_saw1,iw_saw2])
    iw_saw_std = np.nanstd([iw_saw1,iw_saw2])
    iw_naw = np.nanmean([iw_naw1,iw_naw2])
    iw_naw_std = np.nanstd([iw_naw1,iw_naw2])
     
    d = {'SA': [iw_saw, lcw_saw, ucw_saw, smw_saw, sw_saw],
         'NA': [iw_naw, lcw_naw, ucw_naw, smw_naw, sw_naw]}
    df2 = pd.DataFrame(data=d, index=['IW', 'lCW', 'uCW', 'SMW','SW'])

    return(df2)

dom_water_mass_transport = calc_transport_by_water_mass(dom1_22_final,dom1_22_final)
stluc_water_mass_transport = calc_transport_by_water_mass(stluc1_22_final,stluc2_22_final)
stvin_water_mass_transport = calc_transport_by_water_mass(stvin1_22_final,stvin2_22_final)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 10))

plt.tight_layout(pad=0.4, w_pad=6, h_pad=4.5)

font_size = 26

dom_water_mass_transport.plot.barh(stacked=True,ax=axs[0],color=['tab:blue','tab:orange'], edgecolor='black', linewidth=2)
axs[0].set_title('Dominica Passage',fontsize=font_size)
axs[0].set_xlim(-1,0)
axs[0].set_xticks([-1,-0.5,0])
axs[0].tick_params(axis='both', which='major', labelsize=font_size)
axs[0].text(0.005,0.94,'(a)', transform=axs[0].transAxes, size=font_size, weight='bold')
axs[0].get_legend().remove()
axs[0].set_xlabel('Transport [Sv]',fontsize=font_size)

stluc_water_mass_transport.plot.barh(stacked=True,ax=axs[1],color=['tab:blue','tab:orange'], edgecolor='black', linewidth=2)
axs[1].set_title('St. Lucia Passage',fontsize=font_size)
axs[1].set_xlim(-1,0)
axs[1].set_xticks([-1,-0.5,0])
axs[1].tick_params(axis='both', which='major', labelsize=font_size)
axs[1].text(0.005,0.94,'(b)', transform=axs[1].transAxes, size=font_size, weight='bold')
axs[1].get_legend().remove()
axs[1].set_xlabel('Transport [Sv]',fontsize=font_size)

stvin_water_mass_transport.plot.barh(stacked=True,ax=axs[2],color=['tab:blue','tab:orange'], edgecolor='black', linewidth=2)
axs[2].set_title('St. Vincent Passage',fontsize=font_size)
axs[2].set_xlim(-1,0)
axs[2].set_xticks([-1,-0.5,0])
axs[2].tick_params(axis='both', which='major', labelsize=font_size)
axs[2].text(0.005,0.94,'(c)', transform=axs[2].transAxes, size=font_size, weight='bold')
axs[2].get_legend().remove()
axs[2].set_xlabel('Transport [Sv]',fontsize=font_size)
axs[2].legend(fontsize=font_size,markerscale=6,frameon=False,loc='lower left')

hatches = ['','','','','/','','','','','']

for x in np.arange(0,len(axs)):    
        bars = axs[x].patches
        for bar, hatch in zip(bars, hatches):
            bar.set_hatch(hatch)



In [ ]:
dom_water_mass_transport.SA.sum()/(dom_water_mass_transport.SA.sum()+dom_water_mass_transport.NA.sum())

In [ ]:
stluc_water_mass_transport.SA.sum()/(stluc_water_mass_transport.SA.sum()+stluc_water_mass_transport.NA.sum())

In [ ]:
stvin_water_mass_transport.SA.sum()/(stvin_water_mass_transport.SA.sum()+stvin_water_mass_transport.NA.sum())

In [ ]:
dom_water_mass_transport.SA.sum()

In [ ]:
stluc_water_mass_transport.SA.sum()

In [ ]:
stvin_water_mass_transport.SA.sum()

In [ ]:
stvin_water_mass_transport

In [ ]:
dom_water_mass_transport.SA.SMW/(dom_water_mass_transport.SA.SMW+dom_water_mass_transport.NA.SMW)

In [ ]:
stluc_water_mass_transport.SA.SMW/(stluc_water_mass_transport.SA.SMW+stluc_water_mass_transport.NA.SMW)

In [ ]:
stvin_water_mass_transport.SA.SMW/(stvin_water_mass_transport.SA.SMW+stvin_water_mass_transport.NA.SMW)